In [ ]:
!pip install googledrivedownloader;
!pip install fasttext

In [2]:
import fasttext
import numpy as np
import pandas as pd
import pickle
import os

from google_drive_downloader import GoogleDriveDownloader as gdd
from matplotlib import pyplot as plt

In [3]:
def Skipgram(filtered_sentences, ws=3, dim=50):
    model = fasttext.train_unsupervised(filtered_sentences, model='skipgram', ws=ws, dim=dim, neg=5)
    vocab_dict = {} 
    vocab_dict = {word:idx for (idx, word) in enumerate(model.get_words())}
    return model

In [4]:
### Titles
title_trn = pd.read_csv('../preprocessed_data/trn_title.csv', delimiter = ',', names=['title','label'])
title_tst = pd.read_csv('../preprocessed_data/tst_title.csv', delimiter = ',', names=['title','label'])

In [5]:
with open('../preprocessed_data/skipgram_train_lines.txt', 'w') as f:
    for title in list(title_trn['title']):
        for w in title.split(): 
          f.write("%s " % w)
        f.write("\n")

with open('../preprocessed_data/skipgram_test_lines.txt', 'w') as f:
    for title in list(title_tst['title']):
        for w in title.split(): 
          f.write("%s " % w)
        f.write("\n")

In [6]:
skipgram_model = Skipgram('../preprocessed_data/skipgram_train_lines.txt')

In [17]:
f1 = open("../preprocessed_data/skipgram_train_lines.txt", "r")
f2 = open("../preprocessed_data/skipgram_test_lines.txt", "r")

trn_embeddings = []
tst_embeddings = [] 

for line in f1:
    line_stripped = line.strip()  
    trn_embeddings.append(skipgram_model.get_sentence_vector(line_stripped))
f1.close()


for line in f2:
    line_stripped = line.strip()
    tst_embeddings.append(skipgram_model.get_sentence_vector(line_stripped))
f2.close()

if not os.path.exists("../preprocessed_embeddings/"):
    !mkdir ../preprocessed_embeddings

f = open("../preprocessed_embeddings/skipgram_train_embeddings.pkl","wb")
pickle.dump(trn_embeddings,f)
f.close()

f = open("../preprocessed_embeddings/skipgram_test_embeddings.pkl","wb")
pickle.dump(tst_embeddings,f)
f.close()